In [31]:
import os
import json
import glob
import csv
import math
import pandas as pd
from io import StringIO
from pathlib import Path

In [25]:
jars = glob.glob("../versions/*/*.jar")
libs = glob.glob("../versions/*/lib")
versions =  glob.glob("../versions/*")
versions_f = [v.split("\\")[-1] for v in versions]

In [34]:
layer_ex = dict()
ckjm_ex = dict()
for j, l, v, vf in zip(jars, libs, versions, versions_f): 
    process = os.popen(f"java -jar {Path('../layer_extractor.jar')} {j}")
    # layer_ex[vf] = process.read()
    layers = pd.read_csv(StringIO(process.read()))
    process.close()
    if("5.5.13.3" in j):
        # read csv
        ckjm_vals = pd.read_csv(Path("../versions/5.5.13.3/ck-metrics-5.5.13.3.csv"))
    else:
        process = os.popen(f"java -jar {Path('../ckjm_adapted.jar')} {j} {l}")
        ckjm_vals = pd.read_csv(StringIO(process.read()))
    
    ckjm_ex[vf] = ckjm_vals.merge(layers, on='ClassName')
    process.close()

In [35]:
num_classes = [len(a) for a in ckjm_ex.values()]
num_classes

[566, 587, 678, 638, 933, 827]

In [28]:
def fwbr_n(CBO, DIT, WMC, RFC, LCOM):
 return -(9.023 * math.log(CBO + 1) + \
 3.945 * math.log(DIT + 1) - \
 0.678 * math.log(WMC + 1) - \
 0.826 * math.log(RFC + 1) - \
 0.378 * math.log(LCOM + 1))

In [29]:
cols = ['ClassName', 'WMC', 'DIT', 'NOC', 'CBO', 'RFC', 'LCOM', 'CA', 'NPM', 'Layer']
ckjm_dict = {k.split('/')[-1]:pd.DataFrame([x.split(',') for x in v.split('\n')], columns=cols) for k, v in ckjm_ex.items()}

AttributeError: 'DataFrame' object has no attribute 'split'

In [17]:
print(ckjm_dict)
# ckjm_dict['5.0.6']['WMC']

{'../versions/5.0.6':                                              ClassName   WMC   DIT   NOC  \
0                                            ClassName   WMC   DIT   NOC   
1                 com.itextpdf.text.ExceptionConverter    10     4     0   
2                 com.itextpdf.text.pdf.qrcode.Version    13     1     0   
3           com.itextpdf.text.pdf.codec.wmf.MetaObject     3     1     3   
4    com.itextpdf.text.pdf.codec.TiffWriter$FieldImage     1     2     0   
..                                                 ...   ...   ...   ...   
563  com.itextpdf.text.pdf.parser.PdfContentStreamP...     3     1     0   
564  com.itextpdf.text.pdf.PdfWriter$PdfBody$PdfCro...    10     1     0   
565               com.itextpdf.text.pdf.PdfContentByte   176     1     4   
566          com.itextpdf.text.pdf.codec.wmf.MetaState    36     1     0   
567                                                     None  None  None   

      CBO   RFC  LCOM    CA   NPM  
0     CBO   RFC  LCOM    CA  

In [99]:
for version in versions_f[:-1]:
    for metric in cols[1:]:
        print(version, metric, ckjm_dict[version][metric].astype('float64').max() - ckjm_dict[version][metric].astype('float64').min())

5.0.6 WMC 176.0
5.0.6 DIT 5.0
5.0.6 NOC 42.0
5.0.6 CBO 86.0
5.0.6 RFC 436.0
5.0.6 LCOM 9082.0
5.0.6 CA 151.0
5.0.6 NPM 158.0
5.1.3 WMC 176.0
5.1.3 DIT 5.0
5.1.3 NOC 31.0
5.1.3 CBO 86.0
5.1.3 RFC 438.0
5.1.3 LCOM 9533.0
5.1.3 CA 150.0
5.1.3 NPM 158.0
5.2.1 WMC 181.0
5.2.1 DIT 5.0
5.2.1 NOC 31.0
5.2.1 CBO 86.0
5.2.1 RFC 438.0
5.2.1 LCOM 9533.0
5.2.1 CA 157.0
5.2.1 NPM 163.0
5.3.4 WMC 195.0
5.3.4 DIT 5.0
5.3.4 NOC 32.0
5.3.4 CBO 89.0
5.3.4 RFC 455.0
5.3.4 LCOM 11261.0
5.3.4 CA 160.0
5.3.4 NPM 164.0
5.4.5 WMC 208.0
5.4.5 DIT 5.0
5.4.5 NOC 34.0
5.4.5 CBO 94.0
5.4.5 RFC 499.0
5.4.5 LCOM 12393.0
5.4.5 CA 182.0
5.4.5 NPM 170.0


In [114]:
def fwbr_n(CBO, DIT, WMC, RFC, LCOM):
 return -(9.023 * math.log(CBO + 1) + \
 3.945 * math.log(DIT + 1) - \
 0.678 * math.log(WMC + 1) - \
 0.826 * math.log(RFC + 1) - \
 0.378 * math.log(LCOM + 1))

def apply_fwbr_n(row):
    return fwbr_n(row['CBO'], row['DIT'], row['WMC'], row['RFC'], row['LCOM'])

def fwbr(CBO, DIT, WMC, RFC, LCOM, NOC):
 return -(8.753 * math.log(CBO + 1) + \
 2.505 * math.log(DIT + 1) - \
 1.922* math.log(WMC + 1) - \
 0.892 * math.log(RFC + 1) - \
 0.399 * math.log(LCOM + 1) - \
 1.080 * math.log(NOC + 1))

def apply_fwbr(row):
    return fwbr(row['CBO'], row['DIT'], row['WMC'], row['RFC'], row['LCOM'], row['NOC'])


for version in versions_f[:-1]:
    for metric in ['WMC', 'DIT', 'NOC', 'CBO', 'RFC', 'LCOM', 'CA', 'NPM']:
        ckjm_dict[version][metric] = pd.to_numeric(ckjm_dict[version][metric])
    ckjm_dict[version]['fwbr_n'] = ckjm_dict[version].apply(lambda row: apply_fwbr_n(row), axis=1)
    ckjm_dict[version]['fwbr'] = ckjm_dict[version].apply(lambda row: apply_fwbr(row), axis=1)



In [115]:
ckjm_dict['5.0.6']

,ClassName,WMC,DIT,NOC,CBO,RFC,LCOM,CA,NPM,fwbr_n,fwbr
0,com.itextpdf.text.ExceptionConverter,10.0,4.0,0.0,0.0,21.0,3.0,68.0,10.0,-1.646239,3.887454
1,com.itextpdf.text.pdf.qrcode.Version,13.0,1.0,0.0,5.0,27.0,46.0,2.0,8.0,-12.904474,-7.838801
2,com.itextpdf.text.pdf.codec.wmf.MetaObject,3.0,1.0,3.0,0.0,4.0,0.0,5.0,3.0,-0.465162,3.860941
3,com.itextpdf.text.pdf.codec.TiffWriter$FieldImage,1.0,2.0,0.0,1.0,2.0,0.0,1.0,1.0,-9.210885,-6.506950
4,com.itextpdf.text.pdf.parser.LocationTextExtra...,8.0,1.0,0.0,6.0,31.0,14.0,2.0,6.0,-14.916354,-10.373871
...,...,...,...,...,...,...,...,...,...,...,...
562,com.itextpdf.text.pdf.parser.PdfContentStreamP...,3.0,1.0,0.0,6.0,7.0,3.0,1.0,1.0,-17.110867,-13.696434
563,com.itextpdf.text.pdf.PdfWriter$PdfBody$PdfCro...,10.0,1.0,0.0,1.0,22.0,0.0,1.0,6.0,-4.773041,-0.397835
564,com.itextpdf.text.pdf.PdfContentByte,176.0,1.0,4.0,53.0,398.0,5014.0,53.0,158.0,-27.050126,-16.223449
565,com.itextpdf.text.pdf.codec.wmf.MetaState,36.0,1.0,0.0,7.0,69.0,422.0,2.0,36.0,-13.253901,-6.794942
